In [ ]:
import ipywidgets as widgets
from uniswappy import *
from axc_lp import *
from axc_algobot import *
from axc_liquidity import *

tenv = token_scenario_baseline

In [ ]:
import math

nround = 0


def objfunc(x):
    global nround
    print(x)
    #    if (x[0], x[1], x[2], x[3]) in memo:
    #        return memo[(x[0], x[1], x[2], x[3])]
    tenv = token_scenario_baseline
    tenv.samples = 50
    tenv.steps = 500
    tenv.swap_size = 10000
    samples = run_paths(
        tenv,
        [
            [
                [tenv.user_lp, "min_tick", "max_tick"],
                [10.0 ** x[0], 0.98 - x[1], 0.98],
            ]
        ],
        [
            [
                [
                    AlgoBot,
                    AlgoBotParams(
                        price_down=x[2] - x[3],
                        price_down_reset=x[2],
                        price_up=x[4] + x[5],
                        price_up_reset=x[4],
                    ),
                ]
            ]
        ],
    )
    print(np.quantile(samples[0].price, 0.1, axis=0)[-10:-1].mean())
    meandiff = tenv.nav - \
        np.quantile(samples[0].price, 0.10, axis=0)[-50:-1].mean()
    meanhigh = np.quantile(
        samples[0].price, 0.90, axis=0)[-50:-1].mean() - tenv.nav
    #  print(np.quantile(samples[0].price, 0.90, axis=0))
    reserve0low = np.quantile(samples[0].reserve0, 0.1, axis=0)
    reserve0low[reserve0low > 0.0] = 0.0
    reserve0 = reserve0low[-50:-1].mean()
    print(meandiff, meanhigh, reserve0)
    out = (
        (10.0 ** x[0] / 10**4) ** 2.0
        + (meandiff / 0.01) ** 2.0
        + (meanhigh / 0.01) ** 2.0
        + (reserve0 / 10**5) ** 2.0
    )
    print(
        (10.0 ** x[0] / 10**5) ** 2.0,
        (meandiff / 0.01) ** 2.0,
        (meanhigh / 0.01) ** 2.0,
        (reserve0 / 10**5) ** 2.0,
    )
    if nround % 20 == 0:
        plot_samples([samples[0].price])
        plot_samples([samples[0].reserve0])
    print(out)
    #    memo[(x[0], x[1], x[2], x[3])] = out
    nround = nround + 1
    return out


nround = 0

In [ ]:
objfunc([3, 0.48, 0.98, 0.05, 1.02, 0.05])

In [ ]:
#objfunc([5, 0.9])
#[objfunc([x, 0.95]) for x in [3, 4, 5, 5.5]]
nround = 0
[objfunc([3, x, 0.98, 0.05, 1.02, 0.05]) for x in [0.5, 0.7, 0.8, 0.9]]

In [ ]:
from noisyopt import minimizeSPSA
res = minimizeSPSA(objfunc, x0=np.array([5, 0.05, 0.98, 0.05, 1.02, 0.05]), bounds = [[3, 6], [0.01, 0.5], [0.8, 0.99], [0.01, 0.1], [1.01, 1.3], [0.01, 0.1]], niter=5000, paired=False)
res